In [1]:
# @title Requisitos :
%pip install torch torchvision torchaudio
%pip install transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\pedro parentoni\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import os
import glob

# Caminho base do dataset
base_path = "Dataset-news-articles-pt-br-main"

# Mapeia categorias para rótulos numéricos conforme a ordem fornecida
category_labels = {
    "ciencia e tecnologia": 0,
    "economia": 1,
    "entretenimento": 2,
    "esportes": 3,
    "politica": 4
}

# Listas para armazenar textos e rótulos
texts = []
labels = []

# Percorre todas as subpastas dentro do diretório base
for category in os.listdir(base_path):
    category_path = os.path.join(base_path, category)
    
    # Verifica se é um diretório e se a categoria está mapeada
    if os.path.isdir(category_path) and category in category_labels:
        label = category_labels[category]  # Obtém o rótulo numérico da categoria
        
        # Pega todos os arquivos .txt dentro da categoria
        for file_path in glob.glob(os.path.join(category_path, "*.txt")):
            with open(file_path, "r", encoding="utf-8") as file:
                texts.append(file.read())  # Adiciona o texto
                labels.append(label)  # Adiciona o rótulo correspondente

# Exemplo de saída
print(f"Total de textos carregados: {len(texts)}")
print(f"Exemplo de texto: {texts[401][:400]}")
print(f"Rótulo correspondente: {labels[401]} ({list(category_labels.keys())[labels[401]]})")


Total de textos carregados: 500
Exemplo de texto: Autonomia do BC pode ser alterada na Câmara

Aprovado pelo Senado, o projeto de autonomia do Banco Central pode sofrer modificações na Câmara. O partido Novo se prepara para tentar enxugar a proposta que recebeu aval dos senadores para reduzir os chamados acessórios que foram colocados para o BC.

O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessór
Rótulo correspondente: 4 (politica)


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Carregar tokenizer e modelo
model_name = "bert-base-multilingual-cased"  # Escolha o modelo adequado
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

print("Número de textos:", len(texts))
print("Número de labels:", len(labels))


Número de textos: 500
Número de labels: 500


In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.optim import AdamW

# Tokenizar textos com tamanho fixo
max_length = 512  # Limita o número de tokens
inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Converter labels para tensor
labels_tensor = torch.tensor(labels, dtype=torch.long)  # Converte para tensor

# Criar dataset
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels_tensor)

# Definir proporção de treino e teste
train_size = int(0.8 * len(dataset))  # 80% para treino
test_size = len(dataset) - train_size

# Dividir dataset aleatoriamente
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Criar dataloaders para treino e teste
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)

# Definir otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)

# Mover modelo para GPU se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Loop de treinamento
epochs = 10
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Zerar gradientes
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Época {epoch + 1}, Perda: {total_loss:.4f}")

# Avaliação no conjunto de teste
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")


Época 1, Perda: 86.8029
Época 2, Perda: 36.0070
Época 3, Perda: 33.5038
Época 4, Perda: 26.4037
